**What is abstractive summarization?**

Extractive:

Picks important sentences

**Abstractive:**

Understands meaning → rewrites it in new words

Like how a human summarizes.

In [2]:
!git clone https://github.com/lekshmi-j/automatic-text-summarization.git



Cloning into 'automatic-text-summarization'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 44 (delta 16), reused 26 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (44/44), 156.75 KiB | 2.96 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [3]:
!pip install transformers sentencepiece torch


This model was trained on CNN/DailyMail — same domain as your dataset.
So it performs very well.

In [4]:
from transformers import pipeline

summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [5]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")
article = dataset["train"][20]["article"]

summary = summarizer(
    article,
    max_length=150,
    min_length=60,
    do_sample=False
)

print("MODEL SUMMARY:\n", summary[0]["summary_text"])


README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

3.0.0/validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

MODEL SUMMARY:
 NEW: O.J. Simpson will be held without bail after his arrest on robbery charges. NEW: Police say they have no information leading them to believe Simpson was carrying a firearm. Simpson is accused of directing several other men in an alleged armed robbery. Two firearms that police said were used were recovered early Sunday.


**Why chunking is required**

Transformers have token limits (~1024 tokens).
CNN articles are longer.
So we split to chunks and then summarize each chunk and combine.

In [6]:
def chunk_text(text, max_words=500):
    words = text.split()
    for i in range(0, len(words), max_words):
        yield " ".join(words[i:i+max_words])


Extractive vs Abstractive

**Extractive**

Copies original sentences
Always factual
Simple
Fast

**Abstractive**

Generates new sentences
Can hallucinate
Powerful but risky
Compute-heavy

In [7]:
def summarize_article(article_text, max_length=150, min_length=60):
    chunks = chunk_text(article_text)
    summaries = []

    for chunk in chunks:
        summary = summarizer(
            chunk,
            max_length=max_length,
            min_length=min_length,
            do_sample=False
        )
        summaries.append(summary[0]["summary_text"])

    final_summary = " ".join(summaries)
    return final_summary


In [10]:
random_article = """
Up until the 1980s, most natural language processing systems were based on complex sets of hand-written rules. Starting in the late 1980s, however, there was a revolution in natural language processing with the introduction of machine learning algorithms for language processing. This shift was influenced by increasing computational power (see Moore's law) and a decline in the dominance of Chomskyan linguistic theories... (e.g. transformational grammar), whose theoretical underpinnings discouraged the sort of corpus linguistics that underlies the machine-learning approach to language processing.[9]

1990s: Many of the notable early successes in statistical methods in NLP occurred in the field of machine translation, due especially to work at IBM Research, such as IBM alignment models. These systems were able to take advantage of existing multilingual textual corpora that had been produced by the Parliament of Canada and the European Union as a result of laws calling for the translation of all governmental proceedings into all official languages of the corresponding systems of government. However, many systems relied on corpora that were specifically developed for the tasks they were designed to perform. This reliance has been a major limitation to their broader effectiveness and continues to affect similar systems. Consequently, significant research has focused on methods for learning effectively from limited amounts of data.
"""


In [11]:
summary = summarize_article(random_article)
print("SUMMARY:\n")
print(summary)


SUMMARY:

Up until the 1980s, most natural language processing systems were based on complex sets of hand-written rules. Machine learning algorithms for language processing were introduced in the late 1980s. Many of the notable early successes in statistical methods in NLP occurred in the field of machine translation, due especially to work at IBM Research.
